In [1]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

/home/uttaran/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/uttaran/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load training and eval data
def load():
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images  # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images  # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    return train_data,train_labels,eval_data,eval_labels

In [3]:
def Exposure(eval_data,e):
    e = e/255
    for i in range(0,eval_data.shape[0]):
        eval_data[i][0:eval_data.shape[1] - 4]+=e
        eval_data[i] = eval_data[i].clip(min=0,max=1)
    return eval_data

In [4]:
def SIFT(gray,p=0):
    gray = np.array([np.uint8 (x * 255) for x in gray])
    gray = gray.reshape(28,28)
    #gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    kp = sift.detect(gray,None)
    img = cv2.drawKeypoints(gray,kp,None)
    if p:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img

In [5]:
def Translation(eval_data,l):
    temp = np.asarray(eval_data)
    temp = np.reshape(temp,(-1,28,28))
    M = np.float32([[1,0,l],[0,1,0]])
    for i in range(0,eval_data.shape[0]):
        temp[i] = cv2.warpAffine(temp[i],M,(28,28))
        temp[i] = temp[i].clip(min=0,max=1)
    temp = np.reshape(temp,[eval_data.shape[0],eval_data.shape[1]])
    eval_data = temp
    return eval_data

In [6]:
def Rotation(eval_data,angle):
    temp = eval_data.reshape(-1,28,28)
    s,r,c = temp.shape
    M = cv2.getRotationMatrix2D((c/2,r/2),angle,1)
    for i in range (0,s):
        temp[i] = cv2.warpAffine(temp[i],M,(c,r))
        temp[i] = temp[i].clip(min=0,max=1)
    return temp.reshape(-1,eval_data.shape[1])

In [7]:
def Scaling(eval_data,l):
    temp = eval_data.reshape(-1,28,28)
    s,r,c = temp.shape
    for i in range (0,s):
        img = cv2.resize(temp[i],None,fx=l, fy=l, interpolation = cv2.INTER_CUBIC)
        if img.shape[0] > 28:
            m = np.int((img.shape[0]-28)/2)
            temp[i] = img[m:m+28,m:m+28]
        else :
            m = 28
            img = cv2.copyMakeBorder( img, m, m, m, m, cv2.BORDER_CONSTANT)
            m = np.int((img.shape[0]-28)/2)
            temp[i] = img[m:m+28,m:m+28]
        temp[i] = temp[i].clip(min=0,max=1)
    return temp.reshape(-1,eval_data.shape[1])

In [8]:
def Display(eval_data,t):
    if t==0:
        l = np.array(eval_data)
    else :
        l = np.array(eval_data[t])
    l = [int (x * 255) for x in l]
    l = np.asarray(l)
    l = np.reshape(l,(-1,28))
    plt.imshow(l,cmap='gray')

In [9]:
def Plot(eval_data,eval_labels,mnist_classifier):
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    pred = mnist_classifier.predict(input_fn=eval_input_fn)
    my_array = []
    for i, x in enumerate(pred):
        temp = x['probabilities']
        pos = sorted(temp,reverse=True)
        index = [pos.index(v) for v in temp]
        my_array.append(index[eval_labels[i]])
    count = collections.Counter(my_array)
    keys = np.array(list(count.keys()))
    values = np.array(list(count.values()))
    grid = []
    for i in range(0,keys.shape[0]):
        grid.append([keys[i],values[i]])
    grid = np.array(grid)
    grid.view('i8,i8').sort(axis=0)
    plt.plot(grid)

In [10]:
def Test(train_data,train_labels,eval_data,eval_labels,mnist_classifier,p = 0):
    train_eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      shuffle=False)
    eval_result1=mnist_classifier.evaluate(input_fn=train_eval_input_fn)
    print("%%%%%%% Train accuracy %%%%%%%%%%%%\n",eval_result1)
    [a,b,c] = eval_result1.items()

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    eval_result2=mnist_classifier.evaluate(input_fn=eval_input_fn)
    print("######### Test accuracy #############\n",eval_result2)
    [d,e,f] = eval_result2.items()
    if p:
        return a,b,c,d,e,f